## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

path = "c:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc

import platform

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

import sklearn
assert sklearn.__version__ >= "0.20"

# 시드 고정
np.random.seed(21)

## 장르 추천

### 세부 장르 가져오기

In [ ]:
# 파일에서 세부 장르 데이터 가져오기

open_file = open('./text files/detail_genre.txt', 'r', encoding = 'utf8')
text = open_file.read()
open_file.close()

# 파일에서 읽어온 데이터 확인
print(text)

data_list = text.split('\n')

# data_list 의 마지막에 공백이 추가되므로 공백 제거
data_list = data_list[:-2]
# 확인
print(data_list)

### 세부 장르별 단어 가져오기

In [9]:
word_dict_data = {}

# 각 세부 장르 별로 단어 리스트 생성하고 저장
for detail_genre in data_list:
    # 각 세부 장르에 대해 단어 데이터 가져오기
    open_file = open('./text files/' + detail_genre + '_words.txt', 'r', encoding = 'utf8')
    text = open_file.read()
    
    # 파일 닫기
    open_file.close()
    
    # 각 단어 분리하기
    text_list = text.split('\n')
    text_list = text_list[:-2] # 마지막의 공백 제외
    
    # 단어 리스트를 dict 형태로 저장
    word_dict_data[detail_genre] = text_list

In [10]:
word_dict_data['영화-역사']

['땅',
 '기운',
 '점',
 '인간',
 '운명',
 '수',
 '천재',
 '지관',
 '박재상',
 '명당',
 '이용',
 '천하',
 '권력',
 '장동',
 '김씨',
 '가문',
 '계획',
 '가족',
 '후',
 '복수',
 '왕족']

### 세부 장르별 단어 정리

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

In [21]:
# 각 장르별 단어 저장하기

# 장르별 벡터 저장
word_vector_dict = {}

okt = Okt()
vectorizer = CountVectorizer(min_df = 0.05)
    
for detail_genre in data_list:
    contents_tokens = word_dict_data[detail_genre]

    # 벡터화를 위해 단어들을 가지고 문장 생성
    contents_for_vect = []
    sentence = ''
    # 토큰 단위로 구분된 문장을 생성
    for content in contents_tokens:
        sentence += ' ' + content

    # 생성한 문장을 리스트에 추가
    contents_for_vect.append(sentence)
    
    word_vector_dict[detail_genre] = contents_for_vect

In [ ]:
word_vector_dict['TV 시사&교양-기타'][0]

In [20]:
contents_for_vect

[' 아이스크림 모기 탭댄스 버섯 껌 빙판 빨대 식충식물 달팽이 코골 팝콘 수족관 햄 초능력 파리']

## 샘플 줄거리 데이터를 활용해서 거리 계산

In [18]:
from konlpy.tag import Twitter

In [19]:
# 샘플 줄거리 데이터 
new_content = '밀실 안의 살인자, 정유정은 누구인가? 20대 또래 여성을 살해한 정유정의 정체와 범행 동기가 무엇인지에 대해 추적해 본다.'

# 샘플 문장 토큰화
spliter = Twitter()
words = spliter.nouns(new_content)

# 테스트 문장을 피처 벡터화
new_content_vect = vectorizer.transform(words)

C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
okt = Okt()
vectorizer = CountVectorizer(min_df = 0.005)

# 각 세부 장르별 단어 사전을 생성하고 샘플 줄거리 데이터와 비교해서 거리 계산하기
for detail_genre in data_list:
    temp_sentence = ''
    
    # 단어 사전 생성하기
    words_for_dictionary = word_vector_dict[detail_genre]
    splitter = Twitter()
    dictionary = splitter.nouns(words_for_dictionary[0])
    
        
    # 사전 피처 벡터화
    X = vectorizer.fit_transform(dictionary)
    post_vec = X.getrow(0)
    print(post_vec)
    print(type(post_vec))
    
    